# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, Callable, Optional
from ecutilities.core import validate_path, validate_type

import configparser
import numpy as np
import pandas as pd
import subprocess
import sys

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# Notebook setup

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def colab_install_project_code(
    package_name:str # project package name, e.g. git+https://github.com/vtecftwy/metagentools.git@main
):
    """When nb is running on colab, pip install the project code package"""
    try:
        from google.colab import drive
        ON_COLAB = True
        print('The notebook is running on colab')
        print('Installing project code')
        cmd = f"pip install -U {package_name}"
        run(cmd)

    except ModuleNotFoundError:
        ON_COLAB = False
        print('The notebook is running locally, will not automatically install project code')

    return ON_COLAB

When using colab, the project code must be install every time from its github repo or from the Python Package Index (PyPI).

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
colab_install_project_code(package_name='git+https://github.com/vtecftwy/metagentools.git@main')

The notebook is running locally, will not automatically install project code


False

# Display functions

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L64){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L72){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.213184,0.127594,0.598544,2.703998,0.089578
1,-1.432416,0.992436,-1.862047,-0.108099,-0.317770
2,-0.144904,0.948521,-1.822291,0.320756,-0.559796


,0,1,2,3,4,5,6,7,8,9
0,-0.747929,1.348844,0.171768,0.166460,0.866487,0.465583,-1.984931,0.250821,1.470005,-0.477386
1,0.431287,0.004584,-1.053301,-1.298935,-0.258835,-0.379800,-0.177034,-1.338112,-0.274169,2.327783
2,0.951781,-2.812269,-0.962038,0.177506,-1.648600,-0.419919,1.366971,-0.998300,0.425968,1.212941


In [ ]:
#| export
def df_all_cols_and_rows(
    f:Callable,   # function to apply the decorator ti
)-> Callable:     # decorated function
    """decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function"""
    @wraps(f)
    def wrapper(*args, **kwargs):
        max_rows = pd.options.display.max_rows
        max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = None
        pd.options.display.max_columns = None
        f(*args, **kwargs)
        pd.options.display.max_rows = max_rows
        pd.options.display.max_columns = max_cols
    
    return wrapper

In [ ]:
show_doc(df_all_cols_and_rows)

---

### df_all_cols_and_rows

>      df_all_cols_and_rows (f:Callable)

decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| f | Callable | function to apply the decorator ti |
| **Returns** | **Callable** | **decorated function** |

Usage of the decorator

In [ ]:
@df_all_cols_and_rows
def show_df(df):
    display(df)

df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
show_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,85,95,1,57,44,90,90,65,87,4,43,23,23,73,57,91,96,12,55,71,88,92,91,25,63,98,13,94,67,10,91,37,23,31,94,5,6,39,15,85,10,41,94,94,29,49,75,73,95,93
1,33,77,36,96,94,28,8,66,58,17,24,30,3,77,72,42,91,72,25,25,49,19,80,48,64,29,83,28,44,14,65,95,69,41,40,91,52,47,83,90,84,44,45,61,20,59,39,22,74,28
2,9,46,68,61,83,88,41,77,89,97,29,91,23,40,56,4,48,53,91,96,47,79,95,53,27,77,3,56,91,45,47,4,61,95,19,79,95,64,6,88,28,69,79,78,65,2,11,8,10,13


In [ ]:
#| export
@df_all_cols_and_rows
def display_full_df(
    df:pd.DataFrame  # DataFrame to display
):
    """Display a `DataFrame` showing all rows and columns"""
#     if not isinstance(df, pd.DataFrame): raise TypeError('df must me a pandas DataFrame')
    validate_type(df, pd.DataFrame, raise_error=True)
    display(df)

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/eda_stats_utils.py#L38){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df (df:pandas.core.frame.DataFrame)

Display a `DataFrame` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame | DataFrame to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,36,37,12,62,62,12,87,2,70,54,...,22,12,62,77,73,28,27,56,40,88
1,27,75,51,72,7,7,91,60,87,7,...,32,91,27,16,1,30,20,10,52,50
2,58,76,45,52,80,87,49,44,49,76,...,68,78,28,64,7,16,70,81,44,51


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,36,37,12,62,62,12,87,2,70,54,76,16,30,26,44,43,18,74,6,89,1,48,32,88,12,90,44,62,91,71,35,49,1,52,13,39,30,36,76,37,22,12,62,77,73,28,27,56,40,88
1,27,75,51,72,7,7,91,60,87,7,86,16,60,2,43,36,75,59,5,11,6,10,50,36,99,62,22,26,94,9,27,85,8,47,20,62,38,44,89,24,32,91,27,16,1,30,20,10,52,50
2,58,76,45,52,80,87,49,44,49,76,80,63,76,85,33,83,58,16,36,36,22,5,29,5,44,69,6,1,6,0,13,21,57,57,56,19,43,50,73,32,68,78,28,64,7,16,70,81,44,51


# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



In [ ]:
#| hide
nbdev_export()